<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/DeepLab_transfer_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BASICS 
import os
import numpy as np
import torch

# MODEL and TRAIN
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

In [2]:
! git clone https://github.com/mbagci06/lung_covid
! pip install kaggle
! mkdir ~/.kaggle
! cp lung_covid/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d anasmohammedtahir/covidqu

Cloning into 'lung_covid'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 130 (delta 12), reused 0 (delta 0), pack-reused 100
Receiving objects: 100% (130/130), 1.08 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 1.15G/1.15G [00:07<00:00, 213MB/s]
100% 1.15G/1.15G [00:07<00:00, 155MB/s]


In [3]:
!unzip "/content/covidqu.zip" -d "/content/archive/" &> /dev/null

In [4]:
%run lung_covid/functions.ipynb
%run lung_covid/DeepLab.ipynb

In [7]:
# The Calling the validaton dataset
val=Lung_Dataset_RGB(split = "Val",class_folder='lung',shuffle=True,sample_size=700)# The length of the validation dataset to calculate the average value of the loss and accuracy

val_size=val.__len__()
# The Train dataset 
tra=Lung_Dataset_RGB(split = "Train",class_folder='lung',shuffle=True,sample_size=3000)
tra_size=tra.__len__()

# Call cuda, if there is no GPU use CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# The input channel determined already because model is pretrained 
net = DeepLabv3_plus(nInputChannels=3, n_classes=2, os=8, pretrained=True).to(device=device)
# Freezing the back bone part
for param in net.resnet_features.parameters():
  param.requires_grad=False
# The adam optimizer for optimization the values will be same for the other model training
optimizer = optim.Adam(net.parameters(), lr= 0.0001)
# The Loss function Cross entropy the value 
# BCE logistic do not need to normalization to 0 1 
criterion = nn.CrossEntropyLoss()

In [8]:
batch_size=25
# loading images and masks for mini epochs 
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True)

In [9]:
!nvidia-smi

Wed Jun  8 06:36:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |   1399MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
# The number of the epoch 
epoch_number=60
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    for batch_idx, sample in enumerate(data_loader):
        # Train data preparing 
        imgs , true_masks = sample['image'],sample['mask']
        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)
        net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            train_loss += loss.item()
            # Updating the params
            loss.backward()
            optimizer.step()
            # Acc calc train
            acc_train+=binary_acc( masks_pred, true_masks).item()
    for batch_idx, sample in enumerate(data_loader_val):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        net.eval()

        # Call iamge and mask
        imgs , true_masks = sample['image'],sample['mask']

        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            # Validation evaluation 
            masks_pred = net(imgs)
            masks_pred = masks_pred.to(device=device)
            loss_val = criterion(masks_pred, true_masks)
            acc_val +=binary_acc( masks_pred, true_masks).item()
            val_loss += loss_val.item()

    batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3704: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch : 1 / 60
Validation Loss:v 0.07196111977100372
Validation Acc: 0.9697485033954892
Train Acc: 0.9398768603801727
Train  Loss   :t 0.13136858437210322
Epoch : 2 / 60
Validation Loss:v 0.05810244447950806
Validation Acc: 0.9753282240458897
Train Acc: 0.9744925051927567
Train  Loss   :t 0.06039323179672162
Epoch : 3 / 60
Validation Loss:v 0.056956384730126174
Validation Acc: 0.9759604398693357
Train Acc: 0.9802153433362643
Train  Loss   :t 0.04479203829541802
Epoch : 4 / 60
Validation Loss:v 0.04937885661742517
Validation Acc: 0.9786986495767321
Train Acc: 0.9832174335916837
Train  Loss   :t 0.03679398284293711
Epoch : 5 / 60
Validation Loss:v 0.04499182610639504
Validation Acc: 0.9803510563714164
Train Acc: 0.9836454009016354
Train  Loss   :t 0.03549522276346882
Epoch : 6 / 60
Validation Loss:v 0.04660135720457349
Validation Acc: 0.9799501023122242
Train Acc: 0.9848271608352661
Train  Loss   :t 0.03255904344841838
Epoch : 7 / 60
Validation Loss:v 0.04852102816637073
Validation Acc: 

In [11]:
import copy
best_model_wts = copy.deepcopy(net.state_dict())
torch.save({    
    'epoch':40 + 1,
    'model_state_dict':best_model_wts,
    'optimizer_state_dict': optimizer.state_dict(),
    'best_acc': 0.18
},os.path.join(os.getcwd(),'deeplab_transfer_covid_{}_epo.pth'.format(epoch_number)))

In [12]:
# Save loss and acc information to the txt 
textfile = open("deeplab_transfer_covid_{}_epo.txt".format(epoch_number), "w")
biglist=val_loss_array+   train_loss_array+    acc_val_array+    acc_train_array
textfile. write("running_loss_array+   train_loss_array+    acc_val_array+    acc_train_array"+ "\n")
for element in [biglist]:
    textfile. write(str(element) + "\n")
textfile. close()